# Population loss in the immediate aftermath of a nuclear attack

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../src")
from main import Country, run_many_countries

## Single country
Use this to simulate an attack on a single country and visualize the results on an interactive map.

In [ ]:
# country that is attacked
country_name = "Germany" 

# nuclear arsenal with which the country is attacked (number of warheads and yield in kt)
arsenal = 50*[100] 
# arsenal = 50*[15]

In [ ]:
country = Country(country_name, landscan_year=2022, degrade=False, degrade_factor=1)

# sanity check - this corresponds to the population of the selected country
country.data.sum() 

In [ ]:
# country.attack_max_fatality_non_overlapping(arsenal, include_injuries=False)
country.attack_random_non_overlapping(arsenal, include_injuries=False)

In [ ]:
country.get_total_fatalities()

In [ ]:
m = country.plot(show_hit_regions=True, show_population_density=False)
m

## (2) Multiple countries
Once you're confident that everything works for a single country, you can also run the model for multiple countries.

In [ ]:
# This is the SORT scenario of Toon et al. 2008
scenario = {
    "France": 200 * [100],
    "United Kingdom": 200 * [100],
    "Germany": 200 * [100],
    "India": 200 * [100],
    "Pakistan": 200 * [100],
    "Japan": 200 * [100],
    "United States of America": 1000 * [100],
    "China": 1100 * [100],
    "Russia": 1100 * [100],
}

In [ ]:
run_many_countries(scenario, degrade=False, degrade_factor=3, targeting_policy="max_fatality_non_overlapping")